## Ophyd

#### Da documentação oficial:
Ophyd cria representações de hardware em Python como objetos hierárquicos que agrupam valores relacionados do sistema de controle. Essa estrutura permite o Ophyd disponibilizar:

- Uma interface de alto nível consistente para uma ampla gama de dispositivos (usada pelo bluesky).
- Acesso direto de baixo nível ao sistema de controle para debug e desenvolvimento.

Ao apresentar uma interface uniforme, os planos experimentais podem ser independentes dos detalhes do hardware, simplificando a criação desses planos.

#### Como usar?

### Leitura de sinais Read-Only

In [10]:
from ophyd.signal import EpicsSignalRO

In [11]:
i0 = EpicsSignalRO('TEST:DETECTOR:Data_RBV')

#### Aguardar para até realizar a conexão, é possível definir um timeout

In [12]:
i0.wait_for_connection(timeout=1) #Aguarda para conexão com timout definido

#### Ler o sinal com um simples dicionário, retorna o valor e um timestamp

In [13]:
i0.read()

{'TEST:DETECTOR:Data_RBV': {'value': 19.079225223915394,
  'timestamp': 1725460646.720853}}

#### Descrever com mais metadados, como unidade (EGU) e limites

In [14]:
i0.describe()

{'TEST:DETECTOR:Data_RBV': {'source': 'PV:TEST:DETECTOR:Data_RBV',
  'dtype': 'number',
  'shape': [],
  'units': '',
  'lower_ctrl_limit': 0.0,
  'upper_ctrl_limit': 0.0,
  'precision': 0}}

### Dispositivos com permissão de escrita

In [15]:
from ophyd import EpicsSignal

In [16]:
data = EpicsSignal('TEST:MOTORS:m1', name='data')

In [17]:
data.wait_for_connection()

#### Set
É possível utilizar set para realizar ações _sem bloqueio_, isto é, ao realizar data.set(10), o set iniciará o movimento, porém não irá esperar até completar

In [18]:
data.set(0)

Status(obj=EpicsSignal(read_pv='TEST:MOTORS:m1', name='data', timestamp=1725471656.608858, tolerance=1e-05, auto_monitor=False, string=False, write_pv='TEST:MOTORS:m1', limits=False, put_complete=False), done=False, success=False)

#### Set retorna um Status Object
Esses objetos são reponsáveis por notificar quando uma ação "lenta" foi completada. Essa ação pode ser: mover um motor, triggar um detector, aguardar uma temperatura específica. Casa Status possui um timeout associado que pode ser configurado.

In [26]:
status = data.set(10)
print('Mudei data para 10 e não esperei')

Mudei data para 10 e não esperei


In [27]:
status

Status(obj=EpicsSignal(read_pv='TEST:MOTORS:m1', name='data', value=10.0, timestamp=1725471804.372479, tolerance=1e-05, auto_monitor=False, string=False, write_pv='TEST:MOTORS:m1', limits=False, put_complete=False), done=True, success=True)

In [28]:
print('Mudando para outro valor')
status = data.set(-90).wait(timeout=10) #aguarda finalizar, mas tem um timeout de 10s
print('Movimento finalizado')

Mudando para outro valor
Movimento finalizado


CA.Client.Exception...............................................
    Context: "10.15.5.147:5064"
    Source File: ../tcpiiu.cpp line 925
    Current Time: Wed Sep 04 2024 14:56:31.049582266
..................................................................
CA.Client.Exception...............................................
    Context: "10.15.5.147:38979"
    Source File: ../tcpiiu.cpp line 925
    Current Time: Wed Sep 04 2024 14:56:31.051841225
..................................................................


#### Subscribe
É uma maneira de monitorar nosso sinal quando ele mudar

In [10]:
values = []
def monitorador(value, old_value, timestamp, **kwargs):
    values.append({'value': value, 'timestamp': timestamp})


In [11]:
data.subscribe(monitorador)

0

In [15]:
data.set(-100).wait(timeout=10)
data.set(7).wait(timeout=10)
data.set(10).wait(timeout=10)

RuntimeError: Another set() call is still in progress for data

In [13]:
values

[{'value': 10, 'timestamp': 1725471240.422506},
 {'value': 10.0, 'timestamp': 1725471240.557348},
 {'value': 10.0, 'timestamp': 1725471243.327857},
 {'value': 3, 'timestamp': 1725471249.3984149},
 {'value': 3.0, 'timestamp': 1725471249.46474},
 {'value': 7, 'timestamp': 1725471249.4397273},
 {'value': 9, 'timestamp': 1725471249.4456837},
 {'value': 7.0, 'timestamp': 1725471249.473145},
 {'value': 9.0, 'timestamp': 1725471249.47874},
 {'value': 9.0, 'timestamp': 1725471250.050536}]

## Dispositivos mais complexos
É possível unir diversos sinais e dispisitivos em um Device.

In [21]:
from ophyd import Component as Cpt
from ophyd import Device, EpicsSignal, EpicsSignalRO, EpicsMotor, EpicsSignalWithRBV, EpicsMotor, Kind

Considere os seguintes conjuntos de PVs:

Estágios de amostra

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:MOTORS:m1|Motor|Eixo X do estágio da amostra|
|TEST:MOTORS:m2|Motor|Eixo Y do estágio da amostra|
|TEST:MOTORS:m3|Motor|Eixo Z do estágio da amostra|
|TEST:MOTORS:m4|Motor|Eixo de rotação em X do estágio da amostra|
|TEST:MOTORS:m5|Motor|Eixo de rotação em Y do estágio da amostra|
|TEST:MOTORS:m6|Motor|Eixo de rotação em Z do estágio da amostra|
|TEST:MOTORS:m7|Motor|Eixo de translação auxiliar|
|TEST:MOTORS:m8|Motor|Eixo de translação auxiliar|

Detector pontual

|Prefixo|Tipo|Descrição|
|-------|----|---------|
|TEST:DETECTOR:AcquisitionNumber|Setpoint|Sobrescreve o número de aquisições feitas até o momento.|
|TEST:DETECTOR:AcquisitionNumber_RBV|Readback|Indica o número de aquisições realizadas até o momento.|
|TEST:DETECTOR:Trigger|Setpoint|Adquire a leitura atual do detector e salva os dados internamente|
|TEST:DETECTOR:Trigger_RBV|Readback|Indica se uma nova leitura está sendo adquirida (1) ou se já foi adquirida (0).|
|TEST:DETECTOR:Data|Setpoint|Sobrescreve a leitura mais recente com inputs manuais.|
|TEST:DETECTOR:Data_RBV|Readback|Mostra a valor da última leitura do detector.|

In [22]:
class SampleMotor(Device):
    x = Cpt(EpicsMotor, "m1", kind=Kind.hinted)
    """Eixo X do estágio da amostra."""
    y = Cpt(EpicsMotor, "m2", kind=Kind.hinted)
    """Eixo Y do estágio da amostra."""
    z = Cpt(EpicsMotor, "m3", kind=Kind.hinted)
    """Eixo Z do estágio da amostra."""
 
    rx = Cpt(EpicsMotor, "m4")
    """Eixo de rotação em X do estágio da amostra."""
    ry = Cpt(EpicsMotor, "m5")
    """Eixo de rotação em Y do estágio da amostra."""
    rz = Cpt(EpicsMotor, "m6")
    """Eixo de rotação em Z do estágio da amostra."""
 
    utg = Cpt(EpicsMotor, "m7")
    """Eixo de translação auxiliar."""
    uth = Cpt(EpicsMotor, "m8")
    """Eixo de translação auxiliar."""

In [23]:
sample_stage = SampleMotor('TEST:MOTORS:', name='sample_stage')

CA.Client.Exception...............................................
    Context: "Channel: "TEST:MOTORS:m1.HOMF", Connecting to: DESKTOP-EM9GKQH.abtlus.org.br:5064, Ignored: 10.15.5.147:38979"
    Source File: ../cac.cpp line 1320
    Current Time: Wed Sep 04 2024 14:23:05.663324012
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "TEST:MOTORS:m1.HOMR", Connecting to: DESKTOP-EM9GKQH.abtlus.org.br:5064, Ignored: 10.15.5.147:38979"
    Source File: ../cac.cpp line 1320
    Current Time: Wed Sep 04 2024 14:23:05.663340925
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "TEST:MOTORS:m2.RBV", Connecting to: DESKTOP-EM9GKQH.abtlus.org.br:5064, Ignored: 10.15.5.147:38979"
    Source File: ../cac.cpp line 1320
    Current Time: Wed Sep 04 2024 14:23:05.663347492
..................

O atributo kind identifica um sinal que será utilizado em um callback. Por exemplo, em callbacks que realizam plots esse atributo é utilizado para determinar os eixos do plot. Tipos de kind:
- config
- hinted
- normal
- ommited

In [30]:
sample_stage.read()

OrderedDict([('sample_stage_x',
              {'value': 9.0, 'timestamp': 1725470583.455456}),
             ('sample_stage_x_user_setpoint',
              {'value': 9.0, 'timestamp': 1725470583.455456}),
             ('sample_stage_y',
              {'value': 20.0, 'timestamp': 1725456829.013889}),
             ('sample_stage_y_user_setpoint',
              {'value': 20.0, 'timestamp': 1725456829.013889}),
             ('sample_stage_z',
              {'value': 0.0, 'timestamp': 1725451967.754324}),
             ('sample_stage_z_user_setpoint',
              {'value': 0.0, 'timestamp': 1725451967.754324}),
             ('sample_stage_rx',
              {'value': 0.0, 'timestamp': 1725451967.754335}),
             ('sample_stage_rx_user_setpoint',
              {'value': 0.0, 'timestamp': 1725451967.754335}),
             ('sample_stage_ry',
              {'value': 0.0, 'timestamp': 1725451967.754347}),
             ('sample_stage_ry_user_setpoint',
              {'value': 0.0, 'timesta

CA.Client.Exception...............................................
    Context: "DESKTOP-EM9GKQH.abtlus.org.br:46773"
    Source File: ../cac.cpp line 1237
    Current Time: Wed Sep 04 2024 14:24:43.747093917
..................................................................
CA.Client.Exception...............................................
    Context: "DESKTOP-EM9GKQH.abtlus.org.br:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Wed Sep 04 2024 14:25:06.997810163
..................................................................


In [27]:
sample_stage.read_configuration()

OrderedDict([('sample_stage_x_user_offset',
              {'value': 0.0, 'timestamp': 1725453737.879517}),
             ('sample_stage_x_user_offset_dir',
              {'value': 0, 'timestamp': 1725453737.879517}),
             ('sample_stage_x_velocity',
              {'value': 100.0, 'timestamp': 1725453737.879517}),
             ('sample_stage_x_acceleration',
              {'value': 1.0, 'timestamp': 1725453737.879517}),
             ('sample_stage_x_motor_egu',
              {'value': 'degrees', 'timestamp': 1725453737.879517}),
             ('sample_stage_y_user_offset',
              {'value': 0.0, 'timestamp': 1725451967.754311}),
             ('sample_stage_y_user_offset_dir',
              {'value': 0, 'timestamp': 1725451967.754311}),
             ('sample_stage_y_velocity',
              {'value': 100.0, 'timestamp': 1725451967.754311}),
             ('sample_stage_y_acceleration',
              {'value': 1.0, 'timestamp': 1725451967.754311}),
             ('sample_stage_y

In [28]:
sample_stage.x.kind

<Kind.normal|config|hinted: 7>

In [29]:
sample_stage.uth.velocity.kind

<Kind.config: 2>

In [46]:
sample_stage.summary()

data keys (* hints)
-------------------
*sample_stage_rx
 sample_stage_rx_user_setpoint
*sample_stage_ry
 sample_stage_ry_user_setpoint
*sample_stage_rz
 sample_stage_rz_user_setpoint
*sample_stage_utg
 sample_stage_utg_user_setpoint
*sample_stage_uth
 sample_stage_uth_user_setpoint
*sample_stage_x
 sample_stage_x_user_setpoint
*sample_stage_y
 sample_stage_y_user_setpoint
*sample_stage_z
 sample_stage_z_user_setpoint

read attrs
----------
x                    EpicsMotor          ('sample_stage_x')
x.user_readback      EpicsSignalRO       ('sample_stage_x')
x.user_setpoint      EpicsSignal         ('sample_stage_x_user_setpoint')
y                    EpicsMotor          ('sample_stage_y')
y.user_readback      EpicsSignalRO       ('sample_stage_y')
y.user_setpoint      EpicsSignal         ('sample_stage_y_user_setpoint')
z                    EpicsMotor          ('sample_stage_z')
z.user_readback      EpicsSignalRO       ('sample_stage_z')
z.user_setpoint      EpicsSignal         ('samp

#### Para implementar o detector será necessário o uso de um Status Object diferente

In [30]:
from ophyd.status import SubscriptionStatus

### O SubscriptionStatus utiliza eventos Ophyd para se basear se o Status Object pode ser marcado como _finished_

In [31]:
class SampleDetector(Device):
    acquisition_number = Cpt(EpicsSignalWithRBV, "AcquisitionNumber", kind=Kind.omitted)
    """Numero de aquisições realizadas até o momento."""
    trigger_signal = Cpt(EpicsSignalWithRBV, "Trigger", kind=Kind.omitted)
    """Sinal para iniciar uma nova aquisição."""
    data_signal = Cpt(EpicsSignalRO, "Data_RBV", kind=Kind.hinted)
    """Dados da última aquisição."""
 
    def trigger(self):
        super_sts = super().trigger()
 
        # Alternativa: Checar se Trigger_RBV foi para 0.
        def check_value(*, old_value, value, **kwargs):
            return (value == old_value + 1)
 
        sts = SubscriptionStatus(self.acquisition_number, check_value)
        self.trigger_signal.set(1)
 
        return super_sts & sts

Nesse exemplo, o SampleDetector implementa o método trigger (comumente utilizado em scans pelo BlueSky). Esse método é responsável por triggar o detector e reportar (via Status Object) quando o envento de trigger foi completado. O SubscriptionStatus verifica se o valor de AcquitionNumber_RBV é maior que o valor anterior para notificar que a aquisição finalizou

In [39]:
detector = SampleDetector('TEST:DETECTOR:', name='detector')

In [41]:
detector.wait_for_connection(timeout=2)

In [42]:
detector.summary()

data keys (* hints)
-------------------
*detector_data_signal

read attrs
----------
data_signal          EpicsSignalRO       ('detector_data_signal')

config keys
-----------

configuration attrs
-------------------

unused attrs
------------
acquisition_number   EpicsSignalWithRBV  ('detector_acquisition_number')
trigger_signal       EpicsSignalWithRBV  ('detector_trigger_signal')



In [43]:
detector.describe()

OrderedDict([('detector_data_signal',
              {'source': 'PV:TEST:DETECTOR:Data_RBV',
               'dtype': 'number',
               'shape': [],
               'units': '',
               'lower_ctrl_limit': 0.0,
               'upper_ctrl_limit': 0.0,
               'precision': 0})])

In [44]:
detector.read()

OrderedDict([('detector_data_signal',
              {'value': 98.37732362631239, 'timestamp': 1725453892.540498})])

In [45]:
detector.read_configuration()

OrderedDict()

## Sophys Common - https://gitlab.cnpem.br/SOL/bluesky/sophys-common

#### Sophys: Common is a collection of Bluesky and Ophyd utility code (plans, devices, and other useful stuff) for usage at LNLS/SIRIUS beamlines.
#### Sophys stands for **S**irius **Ophy**d and Bluesky utilitie**s**. As an old wise monk once said, nothing beats a cool-looking name.

#### Referências: https://blueskyproject.io/ophyd/